# LSTM first attempt

## Overview

This is my first attempt at creating a neural network that has some knowledge of the order of the data coming into it. 

I'm using a basic LSTM design. It consists of a recurrent neural network that is *unrolled* a number of times. In our case, it is unrolled max_cdr_length/31 times. Each cell has 128 neurons that are fully connected. 

This LSTM then spits out to the final fully connected layer that has weights and a bias. It essentially converts the 128 outputs, down to 124 for our 31 * 4 angles output.

## Details

Some things remain the same. We are still using the same bitfield representation as before (i.e a vector 21 items long with a single '1' where the amino acid should be). We still use the masks but we also introduce the length variable. This allows us to select which of the unrolled steps we are counting as the *last relevant block*. Also, the tensorflow rnn function takes a length parameter so we send in the length here.

I've gone back to batching as this network takes longer to train. I've also introduced *dropout* as a regularisation step, simply because it seems to work apparently. Not sure why though? This means setting the dropout probability at training and validation time.

For some reason, the validation batch has to be the same size as the training one. No idea why but it doesn't seem to matter too much. I choose randomly from the entire set each time.

### Epochs

It turns out we can present the same training data multiple times to the network. Now I'd previously thought this was a bad idea. Turns out, not so much. I can't find any actual data on this though so I think I need to check up on this. However, accuracy did seem to increas a little with more epochs.


## Handy Links

The following came in rather handy:

* https://stackoverflow.com/questions/34670112/tensorflow-rnn-with-varying-length-sentences
* https://towardsdatascience.com/lstm-by-example-using-tensorflow-feb0c1968537
* https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/
* https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/
* https://stackoverflow.com/questions/34670112/tensorflow-rnn-with-varying-length-sentences
* http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
* https://danijar.com/variable-sequence-lengths-in-tensorflow/ * this one is pretty good and covers the important points 


In [ ]:
import sys, os, math, random

import tensorflow as tf
import numpy as np

# Import our shared util
parentdir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
os.sys.path.insert(0,parentdir)
from common.util import *

FLAGS = NNGlobals()
# A higher learning rate seems good as we have few examples in this data set.
# Would that be correct do we think?
FLAGS.learning_rate = 0.35
FLAGS.pickle_filename = 'pdb_martin_03.pickle'
FLAGS.lstm_size = 128   # number of neurons per LSTM cell do we think? 
FLAGS.num_epochs = 5    # number of loops around the training set

# Import common items
parentdir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
os.sys.path.insert(0,parentdir)
from common import gen_data

Functions for the various elements of our graph


In [ ]:
def weight_variable(shape, name ):
  ''' For now I use truncated normals with stdddev of 0.1.'''
  initial = tf.truncated_normal(shape, stddev=0.1, name=name)
  return tf.Variable(initial)

def bias_variable(shape, name):
  initial = tf.constant(1.0, shape=shape, name=name)
  return tf.Variable(initial)

def lstm_cell(size, kprob):
  ''' Return an LSTM Cell or other RNN type cell. We
  have a few choices. We can even throw in a bit of
  dropout if we want.'''

  cell= tf.nn.rnn_cell.BasicLSTMCell(size)
  #cell = tf.nn.rnn_cell.GRUCell(size)
  #cell = tf.nn.rnn_cell.BasicRNNCell(size)
  cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=kprob)
  return cell

In [ ]:
def last_relevant(output, length):
  ''' Taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
  Essentially, we want the last output after the total CDR has been computed.
  That output is then converted to our 4 * max_cdr output. '''
  batch_size = tf.shape(output)[0]
  max_length = tf.shape(output)[1]
  out_size = int(output.get_shape()[2])
  index = tf.range(0, batch_size) * FLAGS.max_cdr_length + (length - 1)
  flat = tf.reshape(output, [-1, out_size])
  relevant = tf.gather(flat, index)
  return relevant


In [ ]:
def create_graph() :
  graph = tf.Graph()

  with tf.device('/gpu:0'):
    with graph.as_default():
      # Input data. We take in padded CDRs but feed in a length / mask as well
      # Apparently the dynamic RNN thingy can cope with variable lengths
      tf_train_dataset = tf.placeholder(tf.int32, [None, FLAGS.max_cdr_length, FLAGS.num_acids],name="train_input") 
      output_size = FLAGS.max_cdr_length * 4
      dmask = tf.placeholder(tf.float32, [None, output_size], name="dmask")
      x = tf.cast(tf_train_dataset, dtype=tf.float32)
      
      # Since we are using dropout, we need to have a placeholder, so we dont set 
      # dropout at validation time
      keep_prob = tf.placeholder(tf.float32, name="keepprob")


      # This is the number of unrolls I think - sequential cells
      # In this example, I'm going for max_cdr_length as we want all the history
      # This will take a while and it is dynamically sized based on the inputs.
      sizes = []
      for i in range(0,FLAGS.max_cdr_length):
        sizes.append(FLAGS.lstm_size)
      rnn_layers = [lstm_cell(size, keep_prob) for size in sizes]
      multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
      
      # 'outputs' is a tensor of shape [batch_size, max_cdr_length, lstm_size]
      # 'state' is a N-tuple where N is the number of LSTMCells containing a
      # tf.contrib.rnn.LSTMStateTuple for each cell
      length = create_length(x)
      outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell, inputs=x, dtype=tf.float32, sequence_length = length)
      relevant  = last_relevant(outputs, length)
      test = tf.placeholder(tf.float32, [None, output_size], name="train_test")

      # Output layer converts our LSTM to 4 outputs (4 angles)
      W_o = weight_variable([FLAGS.lstm_size, output_size], "weight_output")
      b_o = bias_variable([output_size],"bias_output")

      # I use tanh to bound the results between -1 and 1 
      y_conv = tf.tanh( ( tf.matmul(relevant, W_o) + b_o) * dmask, name="output")
      variable_summaries(y_conv, "y_conv")

  return graph

In [ ]:
def create_mask(batch):
  ''' create a mask for our fully connected layer, which
  is a [1] shape that is max_cdr * 4 long.'''
  mask = []
  for model in batch:
    mm = []
    for cdr in model:
      tt = 1
      if not 1 in cdr:
        tt = 0
      for i in range(0,4):
        mm.append(tt)
    mask.append(mm)
  return np.array(mask,dtype=np.float32)

def create_length(batch):
  ''' return the actual lengths of our CDR here. Taken from
  https://danijar.com/variable-sequence-lengths-in-tensorflow/ '''
  used = tf.sign(tf.reduce_max(tf.abs(batch), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length


In [ ]:
def cost(goutput, gtest):
  ''' Our error function which we will try to minimise'''
  # We find the absolute difference between the output angles and the training angles
  # Can't use cross entropy because thats all to do with probabilities and the like
  # Basic error of sum squares diverges to NaN due to gradient so I go with reduce mean
  # Values of -3.0 are the ones we ignore
  # This could go wrong as adding 3.0 to -3.0 is not numerically stable
  mask = tf.sign(tf.add(gtest,3.0))
  basic_error = tf.square(gtest-goutput) * mask
  
  # reduce mean doesnt work here as we just want the numbers where mask is 1
  # We work out the mean ourselves
  basic_error = tf.reduce_sum(basic_error)
  basic_error /= tf.reduce_sum(mask)
  return basic_error


In [ ]:
def run_session(graph, datasets):
  ''' Run the session once we have a graph, training methodology and a dataset '''
  with tf.device('/gpu:0'):
    with tf.Session(graph=graph) as sess:
      training_input, training_output, validate_input, validate_output, test_input, test_output = datasets
      # Pull out the bits of the graph we need
      ginput = graph.get_tensor_by_name("train_input:0")
      gtest = graph.get_tensor_by_name("train_test:0")
      goutput = graph.get_tensor_by_name("output:0")
      gmask = graph.get_tensor_by_name("dmask:0")
      gprob = graph.get_tensor_by_name("keepprob:0")

      # Working out the accuracy
      basic_error = cost(goutput, gtest) 
      # Setup all the logging for tensorboard 
      variable_summaries(basic_error, "Error")
      merged = tf.summary.merge_all() 
      train_writer = tf.summary.FileWriter('./summaries/train',graph)
      
      # So far, I have found Gradient Descent still wins out at the moment
      # https://stackoverflow.com/questions/36162180/gradient-descent-vs-adagrad-vs-momentum-in-tensorflow
      
      #train_step = tf.train.GradientDescentOptimizer(FLAGS.learning_rate).minimize(basic_error)
      train_step = tf.train.AdagradOptimizer(FLAGS.learning_rate).minimize(basic_error) 
      #train_step = tf.train.AdamOptimizer(1e-4).minimize(basic_error)
      #train_step = tf.train.MomentumOptimizer(FLAGS.learning_rate, 0.1).minimize(basic_error)
      
      tf.global_variables_initializer().run()
      print('Initialized')	

      for i in range(0,FLAGS.num_epochs):
        stepnum = 0
        FLAGS.next_batch = 0

        while FLAGS.next_batch < len(training_input):
          batch_is, batch_os = next_batch(training_input, training_output, FLAGS)
          batch_iv, batch_ov = random_batch(validate_input, validate_output, FLAGS)
          mask = create_mask(batch_is)
          summary, _ = sess.run([merged, train_step],
              feed_dict={ginput: batch_is, gtest: batch_os, gmask: mask, gprob: 0.5})
          
          # Find the accuracy at every step, but only print every 100
          # We have to batch here too for some reason? LSTM or something?
          mask = create_mask(batch_iv)
          train_accuracy = basic_error.eval(
              feed_dict={ginput: batch_iv, gtest: batch_ov,  gmask: mask, gprob: 1.0}) 
          
          if stepnum % 10 == 0:
            print('step %d, training accuracy %g' % (stepnum, train_accuracy))
          
          #dm = gmask.eval(feed_dict={ginput: item_is, gtest: item_os, gmask: mask}) 
          #print(dm)
          train_writer.add_summary(summary, stepnum)
          stepnum += 1

      # save our trained net
      saver = tf.train.Saver()
      saver.save(sess, 'saved')

In [ ]:
def run_saved(datasets):
  ''' Load the saved version and then test it against the validation set '''
  with tf.Session() as sess:
    graph = sess.graph
    saver = tf.train.import_meta_graph('saved.meta')
    saver.restore(sess, 'saved')
    training_input, training_output, validate_input, validate_output, test_input, test_output = datasets
    goutput = graph.get_tensor_by_name("output:0")
    ginput = graph.get_tensor_by_name("train_input:0")
    gmask = graph.get_tensor_by_name("dmask:0")
    gprob = graph.get_tensor_by_name("keepprob:0")
    mask = create_mask(validate_input)
    res = sess.run([goutput], feed_dict={ginput: validate_input, gmask: mask, gprob: 1.0})

    # Now lets output a random example and see how close it is, as well as working out the 
    # the difference in mean values. Don't adjust the weights though
    r = random.randint(0, len(validate_input)-1)

    print("Actual              Predicted")
    for i in range(0,len(validate_input[r])):
      sys.stdout.write(bitmask_to_acid(FLAGS, validate_input[r][i]))
      phi = math.degrees(math.atan2(validate_output[r][i*4], validate_output[r][i*4+1]))
      psi = math.degrees(math.atan2(validate_output[r][i*4+2], validate_output[r][i*4+3]))
      sys.stdout.write(": " + "{0:<8}".format("{0:.3f}".format(phi)) + " ")
      sys.stdout.write("{0:<8}".format("{0:.3f}".format(psi)) + " ")
      phi = math.degrees(math.atan2(res[0][r][i*4], res[0][r][i*4+1]))
      psi = math.degrees(math.atan2(res[0][r][i*4+2], res[0][r][i*4+3]))
      sys.stdout.write(" | " + "{0:<8}".format("{0:.3f}".format(phi)) + " ")
      sys.stdout.write("{0:<8}".format("{0:.3f}".format(psi)))  
      print("")

if __name__ == "__main__":

  # If we just want to run the trained net
  if len(sys.argv) > 1:
    if sys.argv[1] == "-r":
      datasets = init_data_sets(FLAGS)
      run_saved(datasets)
      sys.exit()

  datasets = init_data_sets(FLAGS)
  graph = create_graph()
  run_session(graph, datasets)
  run_saved(datasets)
